# Вступление
Две недели назад мы рассмотрели несколько способов использовать объекты при написании программ:

1. "Привязывание" функций для доступа к данным к самим данным. Например, при реализации двоичной кучи мы так инкапсулируем детали реализации.
1. "Мимикрия" под стандартные коллекции при помощи "магических" методов вроде `__getitem__`.
1. Паттерн "итератор", чтобы можно было делать `for` по чему угодно.
1. Паттерн "стратегия", чтобы передавать логику работы (например, стратегию бота) как один объект.
1. Паттерн "команда", чтобы удобнее писать undo.
1. Интерпретатор языка ЯТЬ: создание интерфейса "узел абстрактного синтаксического дерева" с методом `evaluate()`.

В этот раз рассмотрим ещё несколько примеров, которые не очень подчиняются простой аналогии "объект — это данные и код для работы с ними". Для них больше подходит "объект — это штука, которая отвечает за кусок логики в программе". Хотя такое определение, мне кажется, меньше помогает.

# Потоки байт и данных
Первый пример — потоки байт/данных. Простой пример — потоки ввода-вывода. Есть и в Python, и в C++, и в Java, и практически во всех языках.

В процедурном языке вроде Pascal у вас был бы набор процедур: `read(x)` (сокращение для `read(input, x)`), `read(file, x)`, `writeln(2 + 2)`, `close(file)` и прочие. Это несложно и удобно, но может работать только с файлами. Например, если мы захотим "замокать" ввод-вывод, то придётся создавать файл и перенаправлять стандартный поток ввода или вывода в файл.

Почему это проблема? _Отсутствует полиморфизм_. Например, пусть у нас, скажем, есть код для распаковки сжатых данных (вроде [Deflate](https://ru.wikipedia.org/wiki/Deflate)). Откуда могут браться данные?

1. Из строки/массива.
1. Из файла.
1. Из сети.

А распакованные данные также могут идти либо в строку/массив, либо в файл, либо дальше в сеть, либо ещё как-то хитро обрабатываться (например, если у нас там внутри протокол HTTP).

Итого получаем, что есть один алгоритм, но как минимум $3\cdot 3 = 9$ вариаций того, откуда читать данные. Писать алгоритм  девять раз некруто. При этом, например, кодирование "из строки в строку" полезно для тестирования и небольших объёмов. А вот "из сети в файл" звучит как обычное скачивание файла и было бы нехорошо сначала скачивать гигабайт данных из сети в массив, потом распаковывать его в другой массив (итого нам нужно хотя бы два гигабайта оперативной памяти), а распакованные полтора гигабайта писать на диск. Хочется делать "потоково": скачали очередной блок, сразу же распаковали и записали на диск, убрали из памяти.

Для этого обычно используется концепция "потоков". Поток — это что-то, из чего можно что-нибудь читать (или, наоборот, писать). Например, просто байты. Или сразу блоки данных. Или прямо целые сообщения.

Базовый пример — `sys.stdin` в Python. У него есть метод `read()`. Такой же метод есть и у всех файлов, открываемых через `open()`. И можно договориться, что когда у нас появится какой-нибудь способ читать байты (например, из сети), мы делаем это, реализовав метод `read()`. Тогда если у нас есть какой-то код, который работал с `sys.stdin`, ему можно будет подсунуть наш объект, читающий данные из сети, и тот код ничего не заметит (если только он не пользовался чем-то, кроме `read`). Получаем _полиморфизм_: коду всё равно, откуда читать данные, код менять не надо.

In [1]:
def read_dword_be(stream):
    """
    Читает и возвращает из потока знаковое двойное слово (4 байта, как int),
    записанное с использованием big-endian порядка байт
    """
    data = stream.read(4)
    assert len(data) == 4
    # Лучше использовать библиотеку struct, но здесь идёт демонстрация, поэтому никакой магии
    result = 0
    for x in data:
        result = result * 256 + x
    return result

def read_dword_array_be(stream):
    length = read_dword_be(stream)
    result = []
    for _ in range(length):
        result.append(read_dword_be(stream))
    return result

In [2]:
from io import BytesIO

print(read_dword_array_be(BytesIO(bytes([
    0, 0, 0, 2,
    0, 0, 2, 1, # 2 * 256 + 1
    0, 0, 2, 3, # 2 * 256 + 3
]))))

# С таким же успехом можно вызвать read_dword_array_be(sys.stdin.buffer)
# sys.stdin.buffer - это поток _байт_ (не символов)

[513, 515]


А дальше можно здесь накрутить шаблон проектирования ["Декоратор"](https://ru.wikipedia.org/wiki/%D0%94%D0%B5%D0%BA%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80_(%D1%88%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F)) (это не то же самое, что декораторы в Python, просто слово перегружено). Например, давайте реализуем объект, который "декорирует" произвольный поток байт из [RLE](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%B4%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B4%D0%BB%D0%B8%D0%BD_%D1%81%D0%B5%D1%80%D0%B8%D0%B9): сначала читает однобайтовое число `x`, потом байт `c` и декодирует это в $x$ повторов байта `c`.

In [3]:
class RleDecoder:
    def __init__(self, underlying):
        self.underlying = underlying
        self.buffer = bytes()  # Буфер для уже раскодированных байт
        
    def read(self, need_bytes=None):
        # Читаем либо до конца, либо пока не нашли нужное количество байт
        while need_bytes is None or len(self.buffer) < need_bytes:
            read = self.underlying.read(2)
            if not read:  # Если низлежащий поток закончился, то мы тоже закончились
                break
            assert len(read) == 2  # Если в потоке остался только один байт, что-то не так
            x, c = read
            self.buffer += bytes([c]) * x
        if need_bytes is None:
            need_bytes = len(self.buffer)
        result = self.buffer[:need_bytes]
        self.buffer = self.buffer[need_bytes:]
        return result

In [4]:
from io import BytesIO

# Вспомогательная функция для тестов
def rle_decode_bytes(data):
    return RleDecoder(BytesIO(data)).read()

print(rle_decode_bytes(bytes([
    3, ord('x'),
    2, ord('y'),
    1, ord('z')
])))

b'xxxyyz'


In [5]:
# А теперь комбинируем RleDecoder и read_dword_array_be!
print(read_dword_array_be(RleDecoder(BytesIO(bytes([
    3, 0, 1, 5,   # 0 0 0 5
    8, 0,         # 0 0 0 0
                  # 0 0 0 0 
    3, 0, 1, 10,  # 0 0 0 10
    4, 1,         # 0x01010101
    4, 255,       # 0xFFFFFFFF
])))))

[0, 0, 10, 16843009, 4294967295]


Это наглядный пример полиморфизма и комбинирования потоков: мы написали свой `RleDecoder`, а теперь можем его подсовывать везде, где требуется поток ввода. В Python и C++ это не слишком популярно, а вот в языке Java встречается повсеместно. Например, там есть интерфейс [`InputStream`](https://docs.oracle.com/javase/7/docs/api/java/io/InputStream.html), который соответствует "чему-то, из чего можно читать", а есть класс-декоратора [`InflaterInputStream`](https://docs.oracle.com/javase/7/docs/api/java/util/zip/InflaterInputStream.html), который берёт произвольный поток и распаковывает его по методу Deflate в новый поток. Прям как наш `RleDecoder`.

Самая важная мысль: объекты можно выделять не потому что есть какие-то конкретные данные, а потому что удобно выделить некоторую "ответственность" и заключить её в объект. Тогда оказывается, что объекты можно удобно друг с другом комбинировать, если чётко договориться, что именно за "ответственности" и соответствующие им интерфейсы у нас есть.